In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_website(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")

        reviewers_name = []
        ratings = []
        reviews = []
        titles = []
        total_reviews = []

        # Extract business title and total reviews
        title = soup.find('h1', class_='y-css-olzveb').text.strip()
        number_review = soup.find('a', class_='y-css-1x1e1r2').text.strip().replace('(', '').replace(')', '')
        total_rating = soup.find('a', class_='y-css-1x1e1r2')

        #users rating
        li_elements = soup.find_all('li', class_='y-css-1sqelp2')
       

        for li in li_elements:
            reviewers = li.find('a', class_='y-css-1x1e1r2')
            rating = li.find('div', class_='y-css-dnttlc')
            review = li.find('span', class_='raw__09f24__T4Ezm')

            if reviewers and review and rating:
                titles.append(title)
                total_reviews.append(number_review)
                reviewers_name.append(reviewers.get_text(strip=True))  # Extract text safely
                ratings.append(rating.get('aria-label', '').replace(' rating', ''))  # Handle missing rating
                reviews.append(review.get_text(strip=True))  # Extract text safely

        # Print scraped data
        print("Title:", title)
        print("Total Reviews:", number_review)
        print("Reviewers:", reviewers_name)
        print("Ratings:", ratings)
        print("Reviews:", reviews)

        # Store Data in Pandas DataFrame
        data = pd.DataFrame({
            "Title": titles,
            "Total Reviews": total_reviews,
            "Reviewer": reviewers_name,
            "Rating": ratings,
            "Review Text": reviews
        })

        # Save to CSV
        data.to_csv("scraped_data.csv", index=False)
        print("Data successfully saved to scraped_data.csv")

        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

# Target Yelp URL
target_url = "https://www.yelp.ca/biz/pai-northern-thai-kitchen-toronto-5?osq=Restaurants"

# Run the scraper
scraped_data = scrape_website(target_url)

Title: Pai Northern Thai Kitchen
Total Reviews: 3.8k reviews
Reviewers: ['Nhu N.', 'Celton M.', 'Michael C.', 'Lisa D.', 'Angela S.', 'Dom G.', 'Laurel B.', 'Christy Y.', 'Ashley C.', 'Maurice V.']
Ratings: ['5 star', '5 star', '5 star', '4 star', '5 star', '5 star', '5 star', '5 star', '5 star', '5 star']
Reviews: ["It's a really busy restaurant but the staff are kind and patient. I came in with a big group and we ended up being late but the staff were very kind to us. I'm genuinely so grateful that they were accommodating and patient with us. The restaurant is nice, and there's a lot of seating. Service is kind and attentive. There was auto gratuity as we were a big group, and they only accept card for payment. I was able to leave an extra cash tip though- I felt so bad that we were late.I ordered the khao soi with curry- you can't adjust spice level for it. It was really good, but I think I prefer the Uptown location more. The beef was a bit tougher compared to the other one. The fl